In [58]:
# %matplotlib widget

from __future__ import annotations

import re
from math import inf

import matplotlib.colors as mcolors
from test_utilities import test

COLORS = list(mcolors.CSS4_COLORS.keys())

# Type alias
Num = int | float

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 5: Cafeteria ---</h2><p>As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.</p>
<p>You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.</p>
<p>"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.</p>
<p>The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are <em>fresh</em> and which are <span title="No, this puzzle does not take place on Gleba. Why do you ask?"><em>spoiled</em></span>. When you ask how it works, they give you a copy of their database (your puzzle input).</p>
<p>The database operates on <em>ingredient IDs</em>. It consists of a list of <em>fresh ingredient ID ranges</em>, a blank line, and a list of <em>available ingredient IDs</em>. For example:</p>
<pre><code>3-5
10-14
16-20
12-18

1
5
8
11
17
32
</code></pre>

<p>The fresh ID ranges are <em>inclusive</em>: the range <code>3-5</code> means that ingredient IDs <code>3</code>, <code>4</code>, and <code>5</code> are all <em>fresh</em>. The ranges can also <em>overlap</em>; an ingredient ID is fresh if it is in <em>any</em> range.</p>
<p>The Elves are trying to determine which of the <em>available ingredient IDs</em> are <em>fresh</em>. In this example, this is done as follows:</p>
<ul>
<li>Ingredient ID <code>1</code> is spoiled because it does not fall into any range.</li>
<li>Ingredient ID <code>5</code> is <em>fresh</em> because it falls into range <code>3-5</code>.</li>
<li>Ingredient ID <code>8</code> is spoiled.</li>
<li>Ingredient ID <code>11</code> is <em>fresh</em> because it falls into range <code>10-14</code>.</li>
<li>Ingredient ID <code>17</code> is <em>fresh</em> because it falls into range <code>16-20</code> as well as range <code>12-18</code>.</li>
<li>Ingredient ID <code>32</code> is spoiled.</li>
</ul>
<p>So, in this example, <em><code>3</code></em> of the available ingredient IDs are fresh.</p>
<p>Process the database file from the new inventory management system. <em>How many of the available ingredient IDs are fresh?</em></p>
</article>


In [59]:
from itertools import batched


tests: list[dict[str, str | int]] = [
    {
        "name": "Example",
        "s": """
            3-5
            10-14
            16-20
            12-18

            1
            5
            8
            11
            17
            32
        """,
        "expected": 3,
    },
]


@test(tests=tests)
def part_I(s: str) -> int:
    ranges, ids = re.split(r"(?:\r?\n){2,}", s.strip())

    ranges = list(batched(map(int, re.findall(r"\d+", ranges)), n=2))
    ids = map(int, re.findall(r"\d+", ids))

    return sum(1 for id in ids if any(fr <= id <= to for fr, to in ranges))


Test Example passed, for part_I.
Success


In [60]:
with open("../input/day5.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 690


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>690</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.</p>
<p>So that they can stop bugging you when they get new inventory, the Elves would like to know <em>all</em> of the IDs that the <em>fresh ingredient ID ranges</em> consider to be <em>fresh</em>. An ingredient ID is still considered fresh if it is in any range.</p>
<p>Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:</p>
<pre><code>3-5
10-14
16-20
12-18
</code></pre>
<p>The ingredient IDs that these ranges consider to be fresh are <code>3</code>, <code>4</code>, <code>5</code>, <code>10</code>, <code>11</code>, <code>12</code>, <code>13</code>, <code>14</code>, <code>15</code>, <code>16</code>, <code>17</code>, <code>18</code>, <code>19</code>, and <code>20</code>. So, in this example, the fresh ingredient ID ranges consider a total of <em><code>14</code></em> ingredient IDs to be fresh.</p>
<p>Process the database file again. <em>How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?</em></p>
</article>


In [61]:
import bisect
from itertools import batched


tests: list[dict[str, str | int]] = [
    {
        "name": "Example",
        "s": """
            3-5
            10-14
            16-20
            12-18

            1
            5
            8
            11
            17
            32
        """,
        "expected": 14,
    },
]


class OrderedDisjointIntervalTree:
    # genrated with claude and copilot works like an extended standard
    # library ;)

    def __init__(self) -> None:
        # intervals stored as list of (start, end), sorted by start
        self.intervals: list[tuple[Num, Num]] = []

    def _find_left(self, x: int) -> int:
        """Find index of interval with start <= x."""
        i = bisect.bisect_right(self.intervals, (x, inf))
        return i - 1

    def insert(self, a: Num, b: Num) -> None:
        """Insert interval [a, b]."""
        if a > b:
            a, b = b, a

        i = self._find_left(a)  # type: ignore

        # merge with previous if overlapping/touching
        if i >= 0 and self.intervals[i][1] >= a - 1:
            a = min(a, self.intervals[i][0])
            b = max(b, self.intervals[i][1])
            self.intervals.pop(i)
        else:
            i += 1

        # merge with following intervals
        while i < len(self.intervals) and self.intervals[i][0] <= b + 1:
            a = min(a, self.intervals[i][0])
            b = max(b, self.intervals[i][1])
            self.intervals.pop(i)

        # insert merged interval
        self.intervals.insert(i, (a, b))

    def contains_point(self, x: int) -> bool:
        """Check if point x lies in any interval."""
        i = self._find_left(x)
        if i >= 0:
            L, R = self.intervals[i]
            return L <= x <= R
        return False

    def overlaps(self, a: int, b: int) -> bool:
        """Check if interval [a, b] overlaps any stored interval."""
        i = self._find_left(a)
        if i >= 0:
            if self.intervals[i][1] >= a:
                return True

        # check next interval
        if i + 1 < len(self.intervals):
            if self.intervals[i + 1][0] <= b:
                return True

        return False

    def query_range(self, a: int, b: int) -> list[tuple[Num, Num]]:
        """Return all intervals overlapping [a, b]."""
        result: list[tuple[Num, Num]] = []
        i = self._find_left(a)

        # check previous interval
        if i >= 0 and self.intervals[i][1] >= a:
            result.append(self.intervals[i])

        # check following intervals
        i += 1
        while i < len(self.intervals) and self.intervals[i][0] <= b:
            result.append(self.intervals[i])
            i += 1

        return result


@test(tests=tests)
def part_II(s: str) -> int:
    ranges, _ = re.split(r"(?:\r?\n){2,}", s.strip())

    ranges = batched(map(int, re.findall(r"\d+", ranges)), n=2)

    tree = OrderedDisjointIntervalTree()

    to_max = 0
    for start, end in ranges:
        tree.insert(start, end)
        to_max = max(end, to_max)

    total: int = int(sum(end - start + 1 for start, end in tree.query_range(0, to_max)))
    return total


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [62]:
print(f"Part I: {part_II(puzzle)}")

Part I: 344323629240733


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>344323629240733</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
